This is a notebook to cluster and examine the neighborhoods in Toronto as part of the Coursera Capstone Project Week 3 assignment.

The output is a dataframe with the clusters of the neighborhood and to visualize them on a map of Toronto.

First we need to import all the libraries.

In [58]:
# Import numpy and pandas
import numpy as np
import pandas as pd

# install beautifulsoup and lxml
!pip install beautifulsoup4
!pip install lxml

# import beautifulsoup and requests
from bs4 import BeautifulSoup
import requests

# library to handle JSON files
import json

# library to handle requests
import requests

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# install folium
# !conda install -c conda-forge folium=0.5.0 --yes

# map rendering library
import folium

# import Nominatim
from geopy.geocoders import Nominatim

print('All imported correctly')

All imported correctly


This portion includes all the steps to load the postal code values from the website to the dataframe.
(Please refer to P1 - Dataframe - Clean notebook for more details)

In [3]:
# set URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

# gets the source code of the website
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

# prints the source code in a more readable format
# print(soup.prettify())

# create new dataframe with column names
column_names = ['Postcode','Borough','Neighborhood']
table = pd.DataFrame(columns=column_names)

# loops through the webpage source code to find the table entries with 'tr' and 'td' markers and adds them to the dataframe
for entry in soup.find_all('tr', class_=None):
        
    try:
        postcode    =entry.find_all('td', class_=None)[0].text
        borough     =entry.find_all('td', class_=None)[1].text
        neighborhood=entry.find_all('td', class_=None)[2].text
        table = table.append({'Postcode':postcode,'Borough':borough,'Neighborhood':neighborhood},ignore_index=True)

    except:
        continue

# copy table to table2 (for cleaning)
table2=table.copy()

# remove last 3 rows with unrelated data
table2=table2[:-3]

# remove \n from neighborhood values
table2.replace(r'\n','',regex=True,inplace=True)

# copy table2 to table3 for removal of unassigned rows
table3=table2.copy()

# drop unassigned boroughs
table3.drop(table3.index[table3['Borough']=='Not assigned'],inplace=True)

# reset the count of the index so as not to skip the dropped rows
table3.reset_index(drop=True,inplace=True)

# set unassigned neighborhoods to be the same value as that of the borough
for index in table3[table3['Neighborhood']=='Not assigned'].index:
#     print("Value to be replaced: ", table3.loc[index,'Neighborhood'])
#     print("Value to replace with: ", table3.loc[index,'Borough'])
    table3.loc[index,'Neighborhood']=table3.loc[index,'Borough']

# merges neighborhood values for the same postal code and borough
table3=table3.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda tags: ', '.join(tags)).to_frame().reset_index()

# prints the final shape of the dataframe
print("Shape is: ", table3.shape)

# display final table
table3.head(10)

Shape is:  (103, 3)


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


This portion includes all of the steps to get the latitude and longitude values on the dataframe from the csv file.
(Please refer to P2 - Lat & Lng - Clean for more details)

In [4]:
# load the latitude and longitude data from csv file
latlng_file = 'https://cocl.us/Geospatial_data'
latlng = pd.read_csv(latlng_file)
latlng.head()

# remove latitude and longitude columns from dataframe
table3=table3.iloc[:,0:3]

# use merge to get latitude and longitude data on main dataframe
table3 = pd.merge(table3, latlng, how='left',
        left_on='Postcode', right_on='Postal Code')

# remove the duplicate postal code column from csv file
table3.drop('Postal Code',axis=1,inplace=True)

table3.head()

# confirm all columns have data for the same number of rows
table3.describe(include='all')

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103.000000,103.000000
unique,103,11,102,NaN,NaN
top,M5G,North York,Queen's Park,NaN,NaN
freq,1,24,2,NaN,NaN
mean,NaN,NaN,NaN,43.704608,-79.397153
std,NaN,NaN,NaN,0.052463,0.097146
min,NaN,NaN,NaN,43.602414,-79.615819
25%,NaN,NaN,NaN,43.660567,-79.464763
50%,NaN,NaN,NaN,43.696948,-79.388790
75%,NaN,NaN,NaN,43.745320,-79.340923


This portion includes the steps to create a map of Toronto with markers for its various neighborhoods.

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(table3['Latitude'], table3['Longitude'], table3['Borough'], table3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


To get data on nearby venues, we make use of the Foursquare API whose details we define below.

In [16]:
# The code was removed by Watson Studio for sharing.

We then define the function from the previous assignment to get the nearby venues using the Foursquare API.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# toronto venues
toronto_venues = getNearbyVenues(names=table3['Neighborhood'],
                                   latitudes=table3['Latitude'],
                                   longitudes=table3['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [19]:
print(toronto_venues.shape)

(2228, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [21]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",7,7,7,7,7,7
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56


In [22]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
toronto_onehot.shape

(2228, 271)

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,...,0.0,0.0,0.020000,0.00,0.0,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.043478,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.017857,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [28]:
# define function to get common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
# prepare to call function get common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Restaurant,Salad Place,Burger Joint,Bar,Bakery,Sushi Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pharmacy,Comic Shop,Electronics Store,College Stadium
4,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,Gastropub,Gas Station


In [48]:
# exclude Neighborhood column for clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# determine optimum number of clusters
# wcss = []
# for i in range(1, 100):
#     kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
#     kmeans.fit(toronto_grouped_clustering)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 100), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

In [49]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = table3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!
# neighborhoods_venues_sorted.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4.0,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Moving Target,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Department Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Intersection,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Medical Center,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Fried Chicken Joint,Gas Station,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Dim Sum Restaurant,Diner


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers for cluster 0 to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged.loc[toronto_merged['Cluster Labels']==0,'Latitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==0,'Longitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==0,'Neighborhood'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==0,'Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[0],
        fill=True,
        fill_color=rainbow[0],
        fill_opacity=0.7).add_to(map_clusters)

# add markers for cluster 1 to the map
for lat, lon, poi, cluster in zip(toronto_merged.loc[toronto_merged['Cluster Labels']==1,'Latitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==1,'Longitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==1,'Neighborhood'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==1,'Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)    

# add markers for cluster 2 to the map
for lat, lon, poi, cluster in zip(toronto_merged.loc[toronto_merged['Cluster Labels']==2,'Latitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==2,'Longitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==2,'Neighborhood'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==2,'Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[2],
        fill=True,
        fill_color=rainbow[2],
        fill_opacity=0.7).add_to(map_clusters)    

# add markers for cluster 3 to the map
for lat, lon, poi, cluster in zip(toronto_merged.loc[toronto_merged['Cluster Labels']==3,'Latitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==3,'Longitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==3,'Neighborhood'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==3,'Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[3],
        fill=True,
        fill_color=rainbow[3],
        fill_opacity=0.7).add_to(map_clusters)    

# add markers for cluster 4 to the map
for lat, lon, poi, cluster in zip(toronto_merged.loc[toronto_merged['Cluster Labels']==4,'Latitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==4,'Longitude'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==4,'Neighborhood'], 
                                  toronto_merged.loc[toronto_merged['Cluster Labels']==4,'Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[4],
        fill=True,
        fill_color=rainbow[4],
        fill_opacity=0.7).add_to(map_clusters)    
    
map_clusters

Examine cluster 0
In this cluster, there are several parks, playgrounds, and trails.

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,Scarborough Village,0.0,Jewelry Store,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
14,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
23,North York,York Mills West,0.0,Park,Convenience Store,Bank,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Department Store
25,North York,Parkwoods,0.0,Park,Bus Stop,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Event Space
30,North York,"CFB Toronto, Downsview East",0.0,Park,Airport,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
44,Central Toronto,Lawrence Park,0.0,Lawyer,Park,Swim School,Bus Line,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
48,Central Toronto,"Moore Park, Summerhill East",0.0,Park,Playground,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
50,Downtown Toronto,Rosedale,0.0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
64,Central Toronto,"Forest Hill North, Forest Hill West",0.0,Park,Trail,Sushi Restaurant,Jewelry Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
74,York,Caledonia-Fairbanks,0.0,Park,Fast Food Restaurant,Market,Women's Store,Gluten-free Restaurant,Gift Shop,Electronics Store,Eastern European Restaurant,Gourmet Shop,Dumpling Restaurant


We examine cluster 1. In this cluster, there are several convenience stores, drug stores, and dim sum restaurants.

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,York,Weston,1.0,Convenience Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store


Next, we examine cluster 2. In this cluster, there are several cafes or coffee shops, varied food choices from American, Korean, Indian, Turkish, Thai, among others.

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",2.0,Moving Target,Bar,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Department Store
2,Scarborough,"Guildwood, Morningside, West Hill",2.0,Electronics Store,Intersection,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Medical Center,Dim Sum Restaurant,Diner
3,Scarborough,Woburn,2.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
4,Scarborough,Cedarbrae,2.0,Fried Chicken Joint,Gas Station,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Dim Sum Restaurant,Diner
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",2.0,Chinese Restaurant,Convenience Store,Discount Store,Department Store,Coffee Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
7,Scarborough,"Clairlea, Golden Mile, Oakridge",2.0,Bakery,Bus Line,Park,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Dog Run,Diner,Discount Store
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",2.0,American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
9,Scarborough,"Birch Cliff, Cliffside West",2.0,College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
10,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",2.0,Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Furniture / Home Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
11,Scarborough,"Maryvale, Wexford",2.0,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Bakery,Sandwich Place,Vietnamese Restaurant,Shopping Mall,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In cluster 3, there are several baseball fields and food trucks that come along with it.

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,Downsview Central,3.0,Food Truck,Home Service,Baseball Field,Women's Store,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
91,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",3.0,Baseball Field,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
97,North York,"Emery, Humberlea",3.0,Construction & Landscaping,Baseball Field,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


For cluster 4, we find that there are several pizza and fast food places.

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Rouge, Malvern",4.0,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
13,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",4.0,Pizza Place,Pharmacy,Chinese Restaurant,Fried Chicken Joint,Thai Restaurant,Italian Restaurant,Convenience Store,Bank,Fast Food Restaurant,Noodle House
35,East York,"Woodbine Gardens, Parkview Hill",4.0,Fast Food Restaurant,Pizza Place,Bank,Bus Line,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Intersection,Athletics & Sports
40,East York,East Toronto,4.0,Park,Convenience Store,Pizza Place,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
81,York,"The Junction North, Runnymede",4.0,Bus Line,Pizza Place,Brewery,Convenience Store,Caribbean Restaurant,Women's Store,Doner Restaurant,Diner,Discount Store,Dog Run
96,North York,Humber Summit,4.0,Empanada Restaurant,Pizza Place,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
100,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",4.0,Park,Pizza Place,Bus Line,Mobile Phone Shop,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Donut Shop,Deli / Bodega
101,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",4.0,Grocery Store,Pizza Place,Fast Food Restaurant,Beer Store,Sandwich Place,Fried Chicken Joint,Pharmacy,Comic Shop,Electronics Store,College Stadium


That is the end of the assignment. Thank you!